## 選擇其中一個Model
### Speech To Text, STT 
- small

- medium

- large-v2

In [14]:
import whisper

# 加載 Whisper 並讓模型運行在 GPU
model = whisper.load_model("small")
# 轉錄音檔
file_path = "employee_log.m4a"
transcription = model.transcribe(file_path)

# 輸出轉錄結果
print(transcription["text"])


/opt/anaconda3/envs/may4s_env/lib/python3.10/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


我是員工編號EMP123 今天是2025年2月11日早上10點我今天主要負責的專案是AI研發計畫今天完成的模型訓練與部署測試但遇到的問題是GPU記憶體不足明天計畫優化LN推理速度嘗試用量化技術減少記憶體的使用


In [15]:
import whisper

# 加載 Whisper 並讓模型運行在 GPU
model = whisper.load_model("medium")

# 轉錄音檔
file_path = "employee_log.m4a"
transcription = model.transcribe(file_path)

# 輸出轉錄結果
print(transcription["text"])


/opt/anaconda3/envs/may4s_env/lib/python3.10/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


我是員工編號EMP123,今天是2025年2月11日早上10點我今天主要負責的專案是AI研發計畫,今天完成了模型訓練與部署測試但遇到的問題是GPU記憶體不足,明天計畫優化LLM退理速度嘗試用量化技術減少記憶體的使用


In [16]:
import whisper

# 加載 Whisper 並讓模型運行在 GPU
model = whisper.load_model("large-v2")

# 轉錄音檔
file_path = "employee_log.m4a"
transcription = model.transcribe(file_path)

# 輸出轉錄結果
print(transcription["text"])


/opt/anaconda3/envs/may4s_env/lib/python3.10/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


我是員工編號EMP123今天是2025年2月11日早上10點我今天主要負責的專案是AI研發計畫今天完成了模型訓練與部署測試但遇到的問題是GPU記憶體不足明天計畫優化LLM推理速度嘗試用量化技術減少記憶體的使用


## 呼叫 Ollama API

### mistral

In [17]:
import requests
import json
import re
# 連接到 Ollama 伺服器
OLLAMA_URL = "http://localhost:11434/api/generate"

def query_ollama(prompt):
    payload = {
        "model": "mistral:latest",
        "prompt": prompt,
        "stream": False  # 設為 False 讓回應一次返回
    }
    response = requests.post(OLLAMA_URL, json=payload)
    return response.json()["response"]

# 產生 Prompt
prompt = f"""
請根據以下員工日誌內容，解析出專案名稱、員工 ID、時間戳記，並輸出成 JSON 格式：
---
{transcription["text"]}
---
請以 JSON 格式輸出，包含：
- "employee_id"
- "timestamp"
- "project"
- "summary"
"""

# 呼叫 Ollama
output_text = query_ollama(prompt)
print("LLM 輸出結果：", output_text)


# 解析 JSON，去除 LLM 可能產生的多餘文字
match = re.search(r"\{.*\}|\[.*\]", output_text, re.DOTALL)
if match:
    json_text = match.group(0)  # 提取純 JSON
    try:
        parsed_output = json.loads(json_text)
        print("✅ 解析成功：", parsed_output)
    except json.JSONDecodeError as e:
        print("❌ JSON 解析錯誤：", e)
else:
    print("❌ 解析失敗，請檢查 LLM 輸出")


LLM 輸出結果：  {
      "employee_id": "EMP123",
      "timestamp": "2025-02-11T10:00:00",
      "project": "AI 研發計畫",
      "summary": "完成了模型訓練與部署測試，但遇到了 GPU 記憶體不足的問題。明天計劃優化 LLM 推理速度，嘗試使用量化技術來減少記憶體的使用."
   }
✅ 解析成功： {'employee_id': 'EMP123', 'timestamp': '2025-02-11T10:00:00', 'project': 'AI 研發計畫', 'summary': '完成了模型訓練與部署測試，但遇到了 GPU 記憶體不足的問題。明天計劃優化 LLM 推理速度，嘗試使用量化技術來減少記憶體的使用.'}


In [33]:
import pandas as pd

# 轉成 DataFrame
df = pd.DataFrame([parsed_output])

# 顯示表格
print(df)

# 存成 CSV
df.to_csv("employee_log.csv", index=False)


  employee_id            timestamp project  \
0      EMP123  2025-02-11T10:00:00  AI研發計畫   

                                             summary  
0  完成了模型訓練與部署測試但遇到的問題是GPU記憶體不足,明天計畫優化LLM退理速度嘗試用量化...  


### deepseek-r1:8b

In [18]:
import requests
import json

# 連接到 Ollama 伺服器
OLLAMA_URL = "http://localhost:11434/api/generate"

def query_ollama(prompt):
    payload = {
        "model": "deepseek-r1:8b",
        "prompt": prompt,
        "stream": False  # 設為 False 讓回應一次返回
    }
    response = requests.post(OLLAMA_URL, json=payload)
    return response.json()["response"]

# 產生 Prompt
prompt = f"""
請根據以下員工日誌內容，解析出專案名稱、員工 ID、時間戳記，並輸出成 JSON 格式：
---
{transcription["text"]}
---
請以 JSON 格式輸出，包含：
- "employee_id"
- "timestamp"
- "project"
- "summary"
"""

# 呼叫 Ollama
output_text = query_ollama(prompt)
print("LLM 輸出結果：", output_text)

# 解析 JSON，去除 LLM 可能產生的多餘文字
match = re.search(r"\{.*\}|\[.*\]", output_text, re.DOTALL)
if match:
    json_text = match.group(0)  # 提取純 JSON
    try:
        parsed_output = json.loads(json_text)
        print("✅ 解析成功：", parsed_output)
    except json.JSONDecodeError as e:
        print("❌ JSON 解析錯誤：", e)
else:
    print("❌ 解析失敗，請檢查 LLM 輸出")



LLM 輸出結果： <think>
今天，我需要处理用户的一个查询请求。用户提供了一段员工日志，并要求从中解析出专案名称、员工ID和时间戳，然后以特定的JSON格式输出。此外，用户还提到了摘要字段，这可能是他们希望了解当天工作的情况。

首先，我仔细阅读了员工日志的内容。日志中明确提到员工编号是EMP123，日期是2025年2月11日，时间是早上10点。这意味着timestamp字段可以直接提取。专案名称被称为“AI研發計畫”，这应该就是project字段的值。

接下来，用户希望添加一个summary字段，这可能涉及对当天工作情况的总结。根据日志内容，员工完成了模型训练和部署测试，但遇到了GPU内存不足的问题。明天他们计划优化LLM推理速度，并尝试使用量化技术减少内存使用。这说明当天主要是解决前一阶段的问题，并为下一阶段做准备。

因此，summary字段应该简要描述当天的工作内容和遇到的主要问题，以及后续的计划。这样用户就能清楚了解员工当天的工作进展和挑战。

在处理过程中，我需要确保JSON格式正确无误，包含employee_id、timestamp、project和summary四个字段。每个字段都应准确反映日志中的信息，没有遗漏或错误。

最后，我会将提取的信息组合成一个结构化的JSON对象，确保符合用户的要求，并且数据准确可靠。
</think>

```json
{
  "employee_id": "EMP123",
  "timestamp": "2025-02-11T10:00:00",
  "project": "AI研發計畫",
  "summary": "今天完成了模型訓練與部署測試，但遇到的問題是GPU記憶體不足。明天計劃進行專案優化，嘗試使用量化技術減少記憶體的使用以提升LLM推理速度。"
}
```
✅ 解析成功： {'employee_id': 'EMP123', 'timestamp': '2025-02-11T10:00:00', 'project': 'AI研發計畫', 'summary': '今天完成了模型訓練與部署測試，但遇到的問題是GPU記憶體不足。明天計劃進行專案優化，嘗試使用量化技術減少記憶體的使用以提升LLM推理速度。'}


In [ ]:
import pandas as pd

# 轉成 DataFrame
df = pd.DataFrame([parsed_output])

# 顯示表格
print(df)

# 存成 CSV
df.to_csv("employee_log_deepseek.csv", index=False)


  employee_id             timestamp project  \
0      EMP123  2025-02-11T10:00:00Z  AI研發計畫   

                                             summary  
0  今天主要負責AI研發計畫，完成了模型訓練與部署測試，但遇到的問題是GPU記憶體不足。明天計劃...  


### phi4:latest

In [19]:
import requests
import json

# 連接到 Ollama 伺服器
OLLAMA_URL = "http://localhost:11434/api/generate"

def query_ollama(prompt):
    payload = {
        "model": "phi4:latest",
        "prompt": prompt,
        "stream": False  # 設為 False 讓回應一次返回
    }
    response = requests.post(OLLAMA_URL, json=payload)
    return response.json()["response"]

# 產生 Prompt
prompt = f"""
請根據以下員工日誌內容，解析出專案名稱、員工 ID、時間戳記，並輸出成 JSON 格式：
---
{transcription["text"]}
---
請以 JSON 格式輸出，包含：
- "employee_id"
- "timestamp"
- "project"
- "summary"
"""

# 呼叫 Ollama
output_text = query_ollama(prompt)
print("LLM 輸出結果：", output_text)

# 解析 JSON，去除 LLM 可能產生的多餘文字
match = re.search(r"\{.*\}|\[.*\]", output_text, re.DOTALL)
if match:
    json_text = match.group(0)  # 提取純 JSON
    try:
        parsed_output = json.loads(json_text)
        print("✅ 解析成功：", parsed_output)
    except json.JSONDecodeError as e:
        print("❌ JSON 解析錯誤：", e)
else:
    print("❌ 解析失敗，請檢查 LLM 輸出")



LLM 輸出結果： 要解析員工日誌內容並將其轉換成 JSON 格式，我們需要從文字中提取出所需的信息：員工 ID、時間戳記和專案名稱。此外，我們還會提供一個摘要段落以彌補任何可能缺失的細節。以下是如何進行解析：

1. **員工 ID**: 文字中明確指出 "員工編號EMP123"，因此我們可以確定員工 ID 為 `EMP123`。

2. **時間戳記**: 日誌提到日期和時間是 "今天是2025年2月11日早上10點"。這意味著該信息的時間戳記為 `2025-02-11T10:00:00`，假設該描述是當地標準時間。

3. **專案名稱**: 文字中提到 "今天主要負責的專案是AI研發計畫"，因此專案名稱為 `AI 研發計畫`。

4. **摘要**: 日誌中描述了工作內容和遇到的問題。所以我們可以從文字中撰寫一個簡單的摘要：
   - "今天主要完成了模型訓練與部署測試，但在這過程中遇到了GPU記憶體不足的問題。計畫明天優化LLM推理速度並使用量化技術來解決記憶體使用問題。"

結合以上分析，將這些信息整理成 JSON 格式如下：

```json
{
  "employee_id": "EMP123",
  "timestamp": "2025-02-11T10:00:00",
  "project": "AI 研發計畫",
  "summary": "今天主要完成了模型訓練與部署測試，但在這過程中遇到了GPU記憶體不足的問題。計畫明天優化LLM推理速度並使用量化技術來解決記憶體使用問題。"
}
```

這個 JSON 物件包含了員工 ID、時間戳記、專案名稱以及一段簡要的摘要，正確反映出日誌內容中的主要信息。
✅ 解析成功： {'employee_id': 'EMP123', 'timestamp': '2025-02-11T10:00:00', 'project': 'AI 研發計畫', 'summary': '今天主要完成了模型訓練與部署測試，但在這過程中遇到了GPU記憶體不足的問題。計畫明天優化LLM推理速度並使用量化技術來解決記憶體使用問題。'}


In [38]:
import pandas as pd

# 轉成 DataFrame
df = pd.DataFrame([parsed_output])

# 顯示表格
print(df)

# 存成 CSV
df.to_csv("employee_log_phi.csv", index=False)


  employee_id            timestamp project  \
0      EMP123  2025-02-11T10:00:00  AI研發計畫   

                                             summary  
0  今天主要負責完成了模型訓練與部署測試，但遇到GPU記憶體不足的問題。明天計劃優化LLM退理速...  


## ollama chat

```bash
ollama list
ollama run deepseek-r1:8b
```

```bash
根據以下員工日誌內容，解析出專案名稱、員工 ID、時間戳記、總結，並輸出成 JSON 格式：我是員工編號EMP123今天是2025年2月11日早上10點我今天主要負責的專案是AI研發計畫今天完成了模型訓練與部署測試但遇到的問題是GPU記憶體不足明天計畫優化LLM推理速度嘗試用量化技術減少記憶體的使用
```

# 模型表現

## Phi4:latest

根據提供的員工日誌內容，我們可以解析出以下信息：
- **專案名稱**：AI研發計畫
- **員工 ID**：EMP123
- **時間戳記**：2025年2月11日早上10點
- **總結**：
  - 主要負責的專案是AI研發計畫。
  - 完成了模型訓練與部署測試。
  - 遇到的問題是GPU記憶體不足。
  - 明天的計畫包括優化LLM推理速度，並嘗試用量化技術來減少記憶體使用。

以下是將這些信息以 JSON 格式輸出：

```json
{
  "project_name": "AI研發計畫",
  "employee_id": "EMP123",
  "timestamp": "2025-02-11 10:00",
  "summary": {
    "responsibilities": "主要負責的專案是AI研發計畫。",
    "accomplishments": "完成了模型訓練與部署測試。",
    "challenges": "遇到的問題是GPU記憶體不足。",
    "next_steps": "明天計畫優化LLM推理速度，嘗試用量化技術減少記憶體的使用。"
  }
}
```


## deepseek 8b

<think>
嗯，好的，我需要帮助用户将他们的员工日志信息转化为纯JSON格式，并添加一个摘要。首先，我要确认日志中的内容：

“我是員工編號EMP123今天是2025年2月11日早上10點我今天主要負責的專案是AI研發計畫今天完成了模型訓練與部署測試但遇到的問題是GPU記憶體不足明天計畫優化LLM推理速度嘗試用量化技術減少記憶     
體的使用”

从日志中，我可以提取出以下信息：

- 员工ID：EMP123
- 日期和时间：2025年2月11日早上10點
- 专案名称：AI 研发 计划

在用户之前的查询中，我已经将这些信息转换为JSON格式。现在，用户希望添加一个摘要（summary），因此我需要根据日志内容撰写一个简短的总结。

总结应该包括主要事件、遇到的问题以及下一步计划。具体来说：

1. 员工完成了模型训练和部署测量。
2. 遇到了GPU内存不足的问题。
3. 明天计划优化LLM推理速度，尝试使用量化技术减少内存使用。

基于这些信息，我可以写一个摘要，简要说明当天的工作内容、遇到的挑战以及明天的计划。

接下来，我需要将这些整合成一个JSON对象，并添加summary字段。确保整个结构符合用户要求的纯JSON格式，不使用任何Markdown或其他格式。

```json
{
  "employee_id": "EMP123",
  "project_name": "AI 研发 计划",
  "timestamp": "2025年2月11日早上10點",
  "summary": "完成模型训练和部署测量，遇到GPU内存不足的问题，明天计划优化LLM推理速度并尝试使用量化技术减少内存使用。"
}
```

这样，我就将员工日志信息整理成了一个包含employee_id、project_name、timestamp以及summary的JSON对象。摘要部分简要总结了当天的工作内容和下一步计划，符合用户的要求。

在处理这个问题时，我需要确保不遗漏任何重要信息，同时保持数据的准确性和格式的正确性。此外，还要注意JSON的语法是否正确，避免使用非标准的字符或格式。

总结来说，这个过程包括：

1. 理解用户的需求：提取员工ID、专案名称和时间戳，并添加一个摘要。
2. 分析日志内容，提取所需信息。
3. 撰写摘要，涵盖主要事件和解决问题的计划。
4. 整合所有信息，形成正确的JSON格式。

通过这些步骤，我能够准确地满足用户的要求，并提供一个结构清晰、内容完整的响应。
</think>

```json
{
  "employee_id": "EMP123",
  "project_name": "AI 研发 计划",
  "timestamp": "2025年2月11日早上10點",
  "summary": "完成模型训练和部署测量，遇到GPU内存不足的问题，明天计划优化LLM推理速度并尝试使用量化技术减少内存使用。"
}
```